<a href="https://colab.research.google.com/github/yongsa-nut/SF323_CN408_AIEngineer/blob/main/Agentic_Search_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic Search Demo

In [ ]:
#Download files for demo
!wget https://github.com/yongsa-nut/SF323_CN408_AIEngineer/raw/refs/heads/main/my_docs.zip
!unzip my_docs.zip && rm my_docs.zip

In [5]:
# @title Agentic Search
from openai import OpenAI
from google.colab import userdata
from pathlib import Path
import json
import re
import os

class SearchAgent:

    def __init__(self, model = "z-ai/glm-4.6"):
        # LLM setup
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=userdata.get('openrouter'),
        )
        self.model = model

        system_prompt = '''You are a helpful assistant.
        You have accessed to three tools:
        1. read_file
        2. list_file
        3. grep_files
        Use these tools to answer the question about information presented in my_docs folder.
        '''
        self.messages = [{'role':'system','content':system_prompt}]

        self.tools = [{
            "type": "function",
            "function":{
                "name": "read_file",
                "description": "Read a file and return its content as a string",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "The name of the file you want to read."
                        }
                    },
                    "required": ["filename"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "list_folder_contents",
                "description": "List all files and directories in a given folder. Returns a formatted string with directories marked with a trailing slash and items sorted alphabetically.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "folder_path": {
                            "type": "string",
                            "description": "The path to the folder/directory you want to list contents for."
                        }
                    },
                    "required": ["folder_path"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "grep_files",
                "description": "Search for a text pattern within files in a directory (like grep). Returns matching lines with their file paths and line numbers. Searches recursively through all subdirectories.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "pattern": {
                            "type": "string",
                            "description": "The text or regex pattern to search for within files."
                        },
                        "directory": {
                            "type": "string",
                            "description": "The directory path to search in. Defaults to current directory if not specified.",
                            "default": "."
                        },
                        "file_pattern": {
                            "type": "string",
                            "description": "File pattern to filter which files to search (e.g., '*.py' for Python files, '*.txt' for text files, '*' for all files). Defaults to '*' (all files).",
                            "default": "*"
                        },
                        "ignore_case": {
                            "type": "boolean",
                            "description": "Whether to perform case-insensitive search. Defaults to false.",
                            "default": False
                        },
                        "max_results": {
                            "type": "integer",
                            "description": "Maximum number of matching lines to return. Defaults to 100 to prevent overwhelming output.",
                            "default": 100
                        }
                    },
                    "required": ["pattern"]
                }
            }
        }]

    def read_file(self, filename):
        """Read a file and return its content as a string."""
        try:
            with open(filename, 'r') as file:
                return file.read()
        except Exception as e:
            return (f"Error reading file: {e}")

    def list_folder_contents(self, folder_path):
        """List all files and directories in a folder with proper formatting."""
        try:
            items = os.listdir(folder_path)
            result = []

            for item in items:
                full_path = os.path.join(folder_path, item)
                if os.path.isdir(full_path):
                    result.append(f"{item}/")
                else:
                    result.append(item)

            return '\n'.join(sorted(result))

        except Exception as e:
            return f"Error listing directory: {e}"

    def grep_files(self, pattern, directory=".", file_pattern="*", ignore_case=False, max_results=100):
        """
        Search for text pattern in files (like grep)

        Args:
            pattern: Text or regex pattern to search for
            directory: Directory to search in (default: current directory)
            file_pattern: File pattern to search (default: all files)
            ignore_case: Case-insensitive search (default: False)
            max_results: Maximum results to return (default: 100)

        Returns:
            Formatted string with results showing filename:line_number: line_content
        """
        results = []
        flags = re.IGNORECASE if ignore_case else 0

        try:
            regex = re.compile(pattern, flags)
        except re.error as e:
            return f"Error: Invalid regex pattern - {e}"

        try:
            search_path = Path(directory)
            if not search_path.exists():
                return f"Error: Directory '{directory}' does not exist"

            for file_path in search_path.rglob(file_pattern):
                if file_path.is_file():
                    try:
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                            for line_num, line in enumerate(f, 1):
                                if regex.search(line):
                                    results.append(f"{file_path}:{line_num}: {line.rstrip()}")
                                    if len(results) >= max_results:
                                        results.append(f"\n... (truncated at {max_results} results)")
                                        return "\n".join(results)
                    except (PermissionError, OSError):
                        continue  # Skip files we can't read

            if not results:
                return f"No matches found for pattern '{pattern}' in {directory}"

            return "\n".join(results)

        except Exception as e:
            return f"Error during search: {e}"

    def run_tool(self, name, arguments):
        if name == 'read_file':
            return self.read_file(**arguments)
        if name == 'list_folder_contents':
            return self.list_folder_contents(**arguments)
        if name == 'grep_files':
            return self.grep_files(**arguments)

    def run_tools(self, tool_calls):
        tool_results = []
        for tool_call in tool_calls:
            try:
                tool_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                print(f'\033[92mTool\033[0m: {tool_name}({arguments})')
                tool_result = self.run_tool(tool_name, arguments)
                print(f'\033[92mTool Results\033[0m: {tool_result}')
                tool_result = {
                        "role" : "tool",
                        "tool_call_id": tool_call.id,
                        "name":tool_name,
                        "content" : str(tool_result)
                }
            except Exception as e:
                tool_result = {
                        "role" : "tool",
                        "tool_call_id": tool_call.id,
                        "name":tool_name,
                        "content" : f"Error: {e}",
                }
            tool_results.append(tool_result)

        return tool_results

    def run(self):
        while True:
            user_query = input('User: ')
            if user_query.lower() == 'quit':
                break

            self.messages.append({'role': 'user', 'content': user_query})

            # Tool Handling
            while True:
                response = self.client.chat.completions.create(
                    model = self.model,
                    messages = self.messages,
                    tools = self.tools
                )
                if response.choices[0].finish_reason != 'tool_calls':
                    break
                self.messages.append(response.choices[0].message)
                results = self.run_tools(response.choices[0].message.tool_calls)
                self.messages.extend(results)

            # Final response (per turn)
            self.messages.append({'role': 'assistant',
                                  'content': response.choices[0].message.content})
            print('\033[38;5;208mAssistant\033[0m:', response.choices[0].message.content)



In [6]:
# Let's test
agent = SearchAgent()
agent.run()

User: Use grep tool to look up about sonnet 4.5
Tool: grep_files({'pattern': 'sonnet 4.5', 'directory': 'my_docs'})
Tool Results: No matches found for pattern 'sonnet 4.5' in my_docs
Tool: grep_files({'pattern': 'sonnet', 'directory': 'my_docs'})
Tool Results: my_docs/Claude Sonnet 4.5.md:40: - **Claude API**: Direct API access via `claude-sonnet-4-5`
Tool: read_file({'filename': 'my_docs/Claude Sonnet 4.5.md'})
Tool Results: # Claude Sonnet 4.5

Anthropic's most intelligent AI model, announced on September 29, 2025. Described as "the best coding model in the world."

## Overview

Claude Sonnet 4.5 represents a major advancement in AI capabilities, particularly for coding, autonomous operation, and complex multi-step tasks. It's designed to be "more of a colleague" than previous models.

## Key Capabilities

### Coding Excellence
- **SWE-bench Verified**: 77.2% performance
- Capable of building production-ready applications (not just prototypes)
- Best model for coding tasks across the